In [16]:
import json
from tqdm.auto import tqdm

with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [2]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

documents[1]

{'text': 'GitHub - DataTalksClub/data-engineering-zoomcamp#prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?',
 'course': 'data-engineering-zoomcamp'}

In [3]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("multi-qa-distilbert-cos-v1")

In [4]:
user_question = "I just discovered the course. Can I still join it?"

#### Question 1. Getting the embeddings model: First value

In [5]:
vector = model.encode(user_question)
vector[0]

0.078222655

In [6]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [85]:
#documents

In [14]:
ml_docs = []

for d in documents:
    candidate = d["course"] 
    if candidate == "machine-learning-zoomcamp":
        ml_docs.append(d)

len(ml_docs)

375

#### Question 2. Creating the embeddings: Shape

In [17]:
#  create an embedding for both question and answer fields.

embeddings = [] 

for d in tqdm(ml_docs):
    question = d["question"]
    text = d["text"]
    qa_text = f'{question} {text}'
    # compute the embedding for qa_text, append to embeddings
    embeddings.append(model.encode(qa_text))



  0%|          | 0/375 [00:00<?, ?it/s]

In [10]:
import numpy as np
X = np.array(embeddings)

In [11]:
X.shape

(375, 768)

In [12]:
# compute the cosine similarity between the vector and the matrix.

vector.dot(vector)

0.9999999

Question 3. Search: Highest score

In [66]:
scores = X.dot(vector)
max(scores)

0.6506573

### Vector search

#### Question 4. Hit-rate for our search engine 

In [34]:
# compute the similarity between a query vector and all the embeddings.
def hit_rate(relevance_total):
    count = 0

    for line in relevance_total:
        if True in line:
            count = count + 1

    return count / len(relevance_total)

def mrr(relevance_total):
    score = 0 
    for q in relevance_total:
        for rank in range(len(q)):
            if q[rank] == True:
                score += 1/(rank+1)
        
    return score / len(relevance_total)

def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }


class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings
  

    def search(self, v_query, num_results=5):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]



In [19]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [32]:
search_engine = VectorSearchEngine(documents=ml_docs, embeddings=X)
search_engine.search(vector, num_results=5)


[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

In [35]:
def search_function(q):
    v_q = model.encode(q['question'])
    return search_engine.search(v_q, num_results=5)

# evaluate the search engine
results = evaluate(ground_truth, search_function)
results

  0%|          | 0/1830 [00:00<?, ?it/s]

{'hit_rate': 0.9398907103825137, 'mrr': 0.8516484517304189}

#### Question 5. Indexing with Elasticsearch: What's the ID of the document with the highest score?

In [67]:
len(vector)

768

In [39]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
            "text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [71]:
for doc in tqdm(ml_docs):
    question = doc['question']
    text = doc['text']
    qt = question + ' ' + text

    # embeddings; the slowest part in our case
    doc['question_vector'] = model.encode(question)
    doc['text_vector'] = model.encode(text)
    doc['question_text_vector'] = model.encode(qt)

  0%|          | 0/375 [00:00<?, ?it/s]

In [72]:
for doc in tqdm(ml_docs):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/375 [00:00<?, ?it/s]

In [64]:
def elastic_search_knn(field, vector, course):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )


    results = es_results['hits']['hits']
    

    doc_ids = [res['_id'] for res in results]
    scores = [res['_score'] for res in results]
    
    return doc_ids, scores


In [73]:
elastic_search_knn('question_text_vector', vector, 'machine-learning-zoomcamp')

(['QiSBxpABaW2SpJzCzgBx',
  'RSSBxpABaW2SpJzCzgDc',
  'OSSBxpABaW2SpJzCzQA4',
  'pySBxpABaW2SpJzC_wEs',
  'SCSBxpABaW2SpJzCzwBB'],
 [0.82532895, 0.73585373, 0.7295, 0.72849524, 0.7252791])

#### Question 6. Hit-rate for Elasticsearch 

In [83]:
def elastic_search_knn(field, vector, course):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs
    
def text_question_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_knn('question_text_vector', v_q, 'machine-learning-zoomcamp')

In [84]:
evaluate(ground_truth, text_question_vector_knn)

  0%|          | 0/1830 [00:00<?, ?it/s]

{'hit_rate': 0.9398907103825137, 'mrr': 0.8516484517304189}